# Trainer
千帆Python SDK 在使用[resource API实现发起训练微调](./api_based_finetune.ipynb)之外，还提供了Trainer API，可以更方便地实现一体化的训练微调pipeline。同时提供了状态事件回调函数的注册，通过事件分发实现训练流程状态事件的监控。


本例将基于qianfan==0.3.15展示通过Dataset加载本地数据集，并上传到千帆平台，基于ERNIE-Speed-8K进行fine-tune，并使用Model进行批量跑评估数据，直到最终完成服务发布，并最终实现服务调用的完整过程。

In [ ]:
! pip install "qianfan>=0.3.15" -U

In [24]:
import qianfan
qianfan.__version__

'0.3.15'

## 前置准备
- 初始化千帆安全认证AK、SK

In [2]:
import os 

os.environ["QIANFAN_ACCESS_KEY"] = "your_ak"
os.environ["QIANFAN_SECRET_KEY"] = "your_sk"

#### 导入依赖
- `qianfan.trainer.consts` trainer使用中所用到的常量
- `qianfan.resources.console.consts` api层面定义的字段常量
- `qianfan.trainer.configs` trainer使用所需要的config配置数据类
- `qianfan.resources.QfMessages` 用于组装qianfan.ChatCompletion的输入messages
- `qianfan.trainer.LLMFinetune` 大语言模型fine-tune任务Trainer实现
- `qianfan.dataset.Dataset` 千帆dataset类，用于管理千帆平台、本地、第三方数据集的导入导出，数据清洗等操作

In [25]:
from qianfan.trainer.consts import ActionState
from qianfan.model.consts import ServiceType
from qianfan.resources.console import consts as console_consts
from qianfan.trainer.configs import TrainConfig, DatasetConfig
from qianfan.model.configs import DeployConfig
from qianfan.resources import QfMessages
from qianfan.trainer import Finetune
from qianfan.dataset import Dataset
from qianfan.utils import enable_log
import logging

enable_log(logging.INFO)

## 数据集加载

千帆SDK提供了数据集实现帮助我们可以快速的加载本地的数据集到内存，并通过设定DataSource数据源以保存至本地和千帆平台。

In [26]:
from qianfan.dataset import Dataset

# 加载本地数据集
ds: Dataset = Dataset.load(data_file="./data/fin_cqa_train.jsonl")
ds.list()

[INFO][2024-06-14 10:11:20.926] dataset.py:408 [t:8344509248]: no data source was provided, construct
[INFO][2024-06-14 10:11:20.930] dataset.py:276 [t:8344509248]: construct a file data source from path: ./data/fin_cqa_train.jsonl, with args: {}
[INFO][2024-06-14 10:11:20.938] file.py:293 [t:8344509248]: use format type FormatType.Jsonl
[INFO][2024-06-14 10:11:20.950] utils.py:349 [t:8344509248]: start to get memory_map from .qf_cache/dataset/Users/zhonghanjun/pywp/bce-qianfan-sdk/cookbook/finetune/data/fin_cqa_train.arrow
[INFO][2024-06-14 10:11:20.972] utils.py:277 [t:8344509248]: has got a memory-mapped table
[INFO][2024-06-14 10:11:20.984] dataset.py:994 [t:8344509248]: list local dataset data by None


[[{'prompt': '下文中市场价格下降导致市场价格下降事件对应的结果涉及的产品是？在国际奶粉价格下降压力下,国内奶价仍有下降空间',
   'response': [['奶']]}],
 [{'prompt': '下文中市场价格下降导致市场价格下降事件对应的结果涉及的地区是？在国际奶粉价格下降压力下,国内奶价仍有下降空间',
   'response': [['国内']]}],
 [{'prompt': '下文中有哪些因果事件？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['需求增加导致市场价格提升']]}],
 [{'prompt': '下文中需求增加导致市场价格提升事件对应的原因涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['高碳铬铁']]}],
 [{'prompt': '下文中需求增加导致市场价格提升事件对应的原因涉及的行业是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['我们无法得知，可能需要更多内容说明。']]}],
 [{'prompt': '下文中需求增加导致市场价格提升事件对应的结果涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
   'response': [['高碳铬铁']]}],
 [{'prompt': '下文中有哪些因果事件？但由于7-ACA市场价格大幅下跌，跌幅超过50%，导致公司相关原料药销售价格随之下跌',
   'response': [['市场价格下降导致市场价格下降']]}],
 [{'prompt': '下文中市场价格下降导致市场价格下降事件对应的原因涉及的产品是？但由于7-ACA市场价格大幅下跌，跌幅超过50%，导致公司相关原料药销售价格随之下跌',
   'response': [['7-ACA']]}],
 [{'prompt': '下文中市场价格下降导致市场价

从本地数据集上传到BOS

In [27]:
# 保存到千帆平台
from qianfan.dataset.data_source import QianfanDataSource
from qianfan.resources.console import consts as console_consts

bos_bucket_name = "your_bucket_name"
bos_bucket_file_path = "/sdk_ds/"
qianfan_dataset_name = "random_sdk_trainer_ds"

# 创建千帆数据集，并上传保存
qianfan_data_source = QianfanDataSource.create_bare_dataset(
    name=qianfan_dataset_name,
    template_type=console_consts.DataTemplateType.NonSortedConversation,
    storage_type=console_consts.DataStorageType.PrivateBos,
    storage_id=bos_bucket_name,
    storage_path=bos_bucket_file_path,
)

ds = ds.save(qianfan_data_source)

[INFO][2024-06-14 10:11:43.137] baidu_qianfan.py:465 [t:8344509248]: start to create dataset on qianfan
[INFO][2024-06-14 10:11:44.218] baidu_qianfan.py:483 [t:8344509248]: create dataset on qianfan successfully
[INFO][2024-06-14 10:11:44.220] schema.py:36 [t:8344509248]: unpack dataset before validating
[INFO][2024-06-14 10:11:44.223] dataset.py:994 [t:8344509248]: list local dataset data by 0
[INFO][2024-06-14 10:11:44.817] utils.py:465 [t:8344509248]: start to write arrow table to .qf_cache/dataset/.mapper_cache/Users/zhonghanjun/pywp/bce-qianfan-sdk/cookbook/finetune/data/fin_cqa_train_23581f31-9ada-477e-b9c2-1093517f2642.arrow
[INFO][2024-06-14 10:11:44.821] utils.py:481 [t:8344509248]: writing succeeded
[INFO][2024-06-14 10:11:44.821] utils.py:349 [t:8344509248]: start to get memory_map from .qf_cache/dataset/.mapper_cache/Users/zhonghanjun/pywp/bce-qianfan-sdk/cookbook/finetune/data/fin_cqa_train_23581f31-9ada-477e-b9c2-1093517f2642.arrow
[INFO][2024-06-14 10:11:44.822] schema.p

### LLMFinetune 训练
`LLMFinetune` 实现了SFT逻辑的trainer，它内部组装了SFT所需要的基本`Pipeline`, 用于串联数据->训练->模型发布->服务调用等步骤

In [28]:
from qianfan.trainer.consts import PeftType

trainer = Finetune(
    train_type="ERNIE-Speed-8K",
    train_config=TrainConfig(
        epoch=1,
        learning_rate=0.0003,
        max_seq_len=4096,
        peft_type=PeftType.LoRA,
        logging_steps=1,
        warmup_ratio=0.10,
        weight_decay=0.0100,
        lora_rank=8,
        lora_all_linear="True",
    ),
    dataset=DatasetConfig(
        datasets=[ds],
        eval_split_ratio=10,    # 评估集拆分比例 10%
        corpus_proportion=0.03, # 混合千帆通用训练语料 0.03%
    ),
)

### 运行任务
同步运行trainer，训练直到模型发布完成

In [29]:
trainer.run()

[INFO][2024-06-14 10:12:56.543] utils.py:776 [t:6280982528]: data releasing, keep polling
[INFO][2024-06-14 10:12:59.069] utils.py:776 [t:6280982528]: data releasing, keep polling
[INFO][2024-06-14 10:13:01.529] utils.py:776 [t:6280982528]: data releasing, keep polling
[INFO][2024-06-14 10:13:03.986] utils.py:776 [t:6280982528]: data releasing, keep polling
[INFO][2024-06-14 10:13:06.441] utils.py:776 [t:6280982528]: data releasing, keep polling
[INFO][2024-06-14 10:13:08.896] utils.py:783 [t:6280982528]: data releasing succeeded
[INFO][2024-06-14 10:13:11.868] actions.py:663 [t:6280982528]: [train_action] training ... job_name:model0f228692_BxBbC current status: Running, 1% check train task log in https://console.bce.baidu.com/qianfan/train/sft/job-5z1x5c2ecmtx/task-ab2597tkcfud/detail/traininglog
[INFO][2024-06-14 10:13:42.494] actions.py:663 [t:6280982528]: [train_action] training ... job_name:model0f228692_BxBbC current status: Running, 1% check train task log in https://console.bc

获取finetune任务输出：

In [30]:
trainer.output

{'datasets': {'sourceType': 'Platform',
  'versions': [{'versionId': 'ds-wy4hmd811aeh2b3p'}],
  'splitRatio': 10.0,
  'corpusProportion': '0.03%'},
 'task_id': 'task-ab2597tkcfud',
 'job_id': 'job-5z1x5c2ecmtx',
 'metrics': {'BLEU-4': '1.98%',
  'ROUGE-1': '7.41%',
  'ROUGE-2': '0.37%',
  'ROUGE-L': '5.80%'},
 'checkpoints': [],
 'model_id': 'am-eush4fhk3ccb',
 'model_version_id': 'amv-8sgyr4aqptvs',
 'model': <qianfan.model.model.Model at 0x12d8f1bd0>}

### 运行批量评估推理
Model支持模型批量运行评估数据集，并保存到千帆平台

调用Fine-tune得到的Model对象的`batch_run_on_qianfan`发起批量任务，这可能会持续数十分钟

In [31]:
from qianfan.model import Model
from qianfan.dataset import Dataset

# 首先需要先加载测试数据集，这里以加载刚上传的训练集为例子：
test_ds = Dataset.load(qianfan_dataset_id=qianfan_data_source.id)

# 从训练结果中获取模型对象
m: Model = trainer.output["model"]

# 运行批量任务获取结果数据集
result_ds: Dataset = m.batch_inference(test_ds)

[INFO][2024-06-14 11:04:03.995] dataset.py:408 [t:8344509248]: no data source was provided, construct
[INFO][2024-06-14 11:04:03.997] dataset.py:282 [t:8344509248]: construct a qianfan data source from existed id: ds-wy4hmd811aeh2b3p, with args: {}
[INFO][2024-06-14 11:04:04.879] dataset_utils.py:410 [t:8344509248]: start to create evaluation task in model
[INFO][2024-06-14 11:04:05.788] dataset_utils.py:372 [t:8344509248]: start to polling status of evaluation task ame-txnmnuqczdzx
[INFO][2024-06-14 11:04:06.068] dataset_utils.py:379 [t:8344509248]: current eval_state: Pending
[INFO][2024-06-14 11:04:36.344] dataset_utils.py:379 [t:8344509248]: current eval_state: Doing
[INFO][2024-06-14 11:05:06.676] dataset_utils.py:379 [t:8344509248]: current eval_state: Doing
[INFO][2024-06-14 11:05:36.964] dataset_utils.py:379 [t:8344509248]: current eval_state: Doing
[INFO][2024-06-14 11:06:07.290] dataset_utils.py:379 [t:8344509248]: current eval_state: Doing
[INFO][2024-06-14 11:06:37.656] dat

通过这种方式运行完成后，可以直接在本地拿到一份批量运行的结果，我们可以通过dataset.list查看其中的部分数据：

In [32]:
result_ds.list([i for i in range(3,6)])

[INFO][2024-06-14 11:16:28.242] dataset.py:994 [t:8344509248]: list local dataset data by [3, 4, 5]


[{'prompt': '下文中需求增加导致市场价格提升事件对应的结果涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
  'input_prompt': '下文中需求增加导致市场价格提升事件对应的结果涉及的产品是？随着供应缩减，需求回升，高碳铬铁价格企稳回升，预期短期内或仍维持偏强态势，但高碳铬铁产能依然偏向过剩，后期价格弹升空间有限',
  'llm_output': '根据上文描述，涉及的产品是高碳铬铁。',
  'expected_output': '高碳铬铁'},
 {'prompt': '下文中有哪些因果事件？东南亚棕榈油减产,原油价格大幅上涨###东南亚棕榈油减产,原油价格大幅上涨###东南亚棕榈油减产，原油价格大幅上涨###东南亚棕榈油减产，原油价格大幅上涨###东南亚棕榈油减产,原油价格大幅上涨###东南亚棕榈油减产，原油价格大幅上涨###东南亚棕榈油减产,原油价格大幅上涨###东南亚棕榈油减产,原油价格大幅上涨###东南亚棕榈油减产,原油价格大幅上涨',
  'input_prompt': '下文中有哪些因果事件？东南亚棕榈油减产,原油价格大幅上涨###东南亚棕榈油减产,原油价格大幅上涨###东南亚棕榈油减产，原油价格大幅上涨###东南亚棕榈油减产，原油价格大幅上涨###东南亚棕榈油减产,原油价格大幅上涨###东南亚棕榈油减产，原油价格大幅上涨###东南亚棕榈油减产,原油价格大幅上涨###东南亚棕榈油减产,原油价格大幅上涨###东南亚棕榈油减产,原油价格大幅上涨',
  'llm_output': '上述文本描述了东南亚棕榈油减产和原油价格大幅上涨这两个事件，并且它们之间存在因果关系。文本中多次提到这两个事件，表达的意思是东南亚棕榈油减产导致了原油价格的上涨。因此，文本中的因果事件是：东南亚棕榈油减产导致了原油价格的大幅上涨。',
  'expected_output': '供给减少导致市场价格提升'},
 {'prompt': '下文中市场价格下降导致销量（消费）减少事件对应的原因涉及的产品是？而发改委分别在5月10日和6月9日对国内汽柴油价格进行了下调，汽油和柴油零售价格分别累计下调了860和820元/吨，导致石化双雄（尤其是中石化）不仅炼油损逐月扩大

在完成模型的批量运行后，我们可以对模型有一个简单的体感评估，如果效果不错，我们可以选择发布成服务以最终应用生产：

In [33]:
#-# cell_skip
from qianfan.model import Service
from qianfan.model.consts import ServiceType
from qianfan.resources.console.consts import DeployPoolType

sft_svc: Service = m.deploy(DeployConfig(
    name="spcorpus",
    endpoint_suffix="sdkcorpus",
    replicas=1, # 副本数， 与qps强绑定
    pool_type=DeployPoolType.PrivateResource, # 私有资源池
    service_type=ServiceType.Chat,
    hours=1,
))


[INFO][2024-06-14 11:16:56.899] model.py:518 [t:8344509248]: ready to deploy service with model am-eush4fhk3ccb/amv-8sgyr4aqptvs
[INFO][2024-06-14 11:22:07.557] model.py:575 [t:8344509248]: service svco-nxavbjiyqanc has been deployed in `o0luzbiw_sdkcorpus` 


使用Finetune之后的模型服务和原始的预置模型服务调用：

In [34]:
#-# cell_skip
from qianfan import ChatCompletion
### 使用Model & Service调用模型

problem="下文中有哪些因果事件？无取向硅钢广泛应用于铁芯等电机零部件其产量的持续提升导致市场竞争愈发激烈，价格进一步降低，从而有效减少新能源汽车驱动电机行业内企业的成本支出"

#获取服务对象，即ChatCompletion等类型的对象
chat_comp: ChatCompletion = sft_svc.get_res()
sft_chat_resp = chat_comp.do(messages=[{"content": problem, "role": "user"}])
sft_chat_resp["result"]

'下文中的因果事件如下：\n\n1. 无取向硅钢广泛应用于铁芯等电机零部件。\n2. 无取向硅钢产量的持续提升导致市场竞争愈发激烈。\n3. 市场竞争激烈导致无取向硅钢价格进一步降低。\n4. 无取向硅钢价格降低有效减少新能源汽车驱动电机行业内企业的成本支出。'